In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dlproj/B_test.csv
/kaggle/input/dlproj/B_train.csv
/kaggle/input/dlproj/S_train.csv
/kaggle/input/dlproj/S_test.csv
/kaggle/input/dlproj/T_train.csv
/kaggle/input/dlproj/T_test.csv
/kaggle/input/dlproj/G_test.csv
/kaggle/input/dlproj/G_train.csv


# **Importing Modules**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pandas import DataFrame , concat
from sklearn.metrics import mean_absolute_error , mean_squared_error
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from numpy import mean , concatenate
from math import sqrt
from pandas import read_csv
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Activation

from sklearn.preprocessing import LabelEncoder
from numpy import array , hstack
from tensorflow import keras
import tensorflow as tf

In [3]:
dataset = pd.read_csv("/kaggle/input/dlproj/S_train.csv", header=0, index_col=0)
t = dataset.columns.tolist()

dataset = dataset[['weather', 'temperature', 'pressure', 'humidity', 'wind_speed', 'wind_direction', 'PM10_Concentration', 'NO2_Concentration', 'CO_Concentration', 'O3_Concentration', 'SO2_Concentration', 'PM25_Concentration']]

In [4]:
dataset

,weather,temperature,pressure,humidity,wind_speed,wind_direction,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration,PM25_Concentration
2014-10-29 11:00:00,1.0,0.794118,0.540984,0.53,0.034722,13.0,0.083333,0.111111,0.057218,0.193460,0.014028,29.0
2014-10-29 12:00:00,1.0,0.803922,0.524590,0.53,0.055556,13.0,0.050505,0.068376,0.057975,0.201635,0.010020,29.0
2014-10-29 13:00:00,1.0,0.811765,0.508197,0.52,0.034722,13.0,0.050505,0.055556,0.054005,0.204360,0.010020,28.0
2014-10-29 14:00:00,1.0,0.819608,0.508197,0.50,0.041667,13.0,0.039141,0.059829,0.052618,0.188011,0.008016,26.0
2014-10-29 15:00:00,1.0,0.819608,0.491803,0.50,0.037037,13.0,0.075758,0.076923,0.053627,0.190736,0.010020,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2015-03-25 01:00:00,2.0,0.576471,0.647541,0.76,0.034722,13.0,0.021465,0.034188,0.037810,0.272480,0.006012,17.0
2015-03-25 02:00:00,2.0,0.576471,0.639344,0.75,0.020833,13.0,0.031566,0.029915,0.038440,0.253406,0.008016,18.0
2015-03-25 03:00:00,2.0,0.576471,0.627869,0.74,0.020833,13.0,0.035354,0.029915,0.039700,0.242507,0.010020,20.0
2015-03-25 04:00:00,2.0,0.574510,0.626230,0.74,0.013889,14.0,0.061869,0.034188,0.039700,0.234332,0.010020,21.0


# **Data Pre-Processing**

In [5]:
# Encode Categorical Values If any
dataset= dataset.values
encoder = LabelEncoder()
dataset[:,3] = encoder.fit_transform(dataset[:,3])
# Assigning Feature Names
dataset = pd.DataFrame(dataset)
dataset.columns = ['weather', 'temperature', 'pressure', 'humidity', 'wind_speed', 'wind_direction', 'PM10_Concentration', 'NO2_Concentration', 'CO_Concentration', 'O3_Concentration', 'SO2_Concentration', 'PM25_Concentration']

In [6]:
#Data Pre-processing step
x_1 = dataset['weather'].values
x_2 = dataset['temperature'].values
x_3 = dataset['pressure'].values
x_4 = dataset['humidity'].values
x_5 = dataset['wind_speed'].values
x_6 = dataset['wind_direction'].values
x_7 = dataset['PM10_Concentration'].values
x_8 = dataset['NO2_Concentration'].values
x_9 = dataset['CO_Concentration'].values
x_10 = dataset['O3_Concentration'].values
x_11 = dataset['SO2_Concentration'].values
y = dataset['PM25_Concentration'].values

# Convert to [rows, columns] structure
x_1 = x_1.reshape((len(x_1), 1))
x_2 = x_2.reshape((len(x_2), 1))
x_3 = x_3.reshape((len(x_3), 1))
x_4 = x_4.reshape((len(x_4), 1))
x_5 = x_5.reshape((len(x_5), 1))
x_6 = x_6.reshape((len(x_6), 1))
x_7 = x_7.reshape((len(x_7), 1))
x_8 = x_8.reshape((len(x_8), 1))
x_9 = x_9.reshape((len(x_9), 1))
x_10 = x_10.reshape((len(x_10), 1))
x_11 = x_11.reshape((len(x_11), 1))
y = y.reshape((len(y), 1))
print ("x_1.shape" , x_1.shape) 
print ("x_2.shape" , x_2.shape) 
print ("y.shape" , y.shape)

# Normalization 
scaler = MinMaxScaler(feature_range=(0, 1))
x_1_scaled = scaler.fit_transform(x_1)
x_2_scaled = scaler.fit_transform(x_2)
x_3_scaled = scaler.fit_transform(x_3)
x_4_scaled = scaler.fit_transform(x_4)
x_5_scaled = scaler.fit_transform(x_5)
x_6_scaled = scaler.fit_transform(x_6)
x_7_scaled = scaler.fit_transform(x_7)
x_8_scaled = scaler.fit_transform(x_8)
x_9_scaled = scaler.fit_transform(x_9)
x_10_scaled = scaler.fit_transform(x_10)
x_11_scaled = scaler.fit_transform(x_11)
y_scaled = scaler.fit_transform(y)

# Horizontally stack columns
dataset_stacked = hstack((x_1_scaled, x_2_scaled,x_2_scaled, x_3_scaled, x_4_scaled, x_5_scaled,x_7_scaled,x_8_scaled, x_9_scaled,x_10_scaled,x_11_scaled, y_scaled))
print ("dataset_stacked.shape" , dataset_stacked.shape)

x_1.shape (3523, 1)
x_2.shape (3523, 1)
y.shape (3523, 1)
dataset_stacked.shape (3523, 12)


**Function to Get Window size Data** 

In [7]:
def split_sequences(sequences, n_steps_in, n_steps_out):
 X, y = list(), list()
 for i in range(len(sequences)):
  end_ix = i + n_steps_in
  out_end_ix = end_ix + n_steps_out-1
  # check if we are beyond the dataset
  if out_end_ix > len(sequences):
   break
  # gather input and output parts of the pattern
  seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
  X.append(seq_x)
  y.append(seq_y)
 return array(X), array(y)
n_steps_in, n_steps_out = 128 , 30
# covert into input/output
X, y = split_sequences(dataset_stacked, n_steps_in, n_steps_out)
print ("X.shape" , X.shape) 
print ("y.shape" , y.shape)

X.shape (3367, 128, 11)
y.shape (3367, 30)


**Early Stopping Mechanism**

In [8]:
from tensorflow.keras.callbacks import EarlyStopping
callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.00001,
    patience=20,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False
)

# **Model Defination**

In [9]:
from sklearn.model_selection import train_test_split
train_X, test_X,train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 0)


train_X.shape 
train_y.shape 
test_X.shape 
test_y.shape 
n_features = 11


opt = keras.optimizers.Adam(learning_rate=0.0001)
# define model
model = Sequential() #
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(n_steps_out))
model.add(Activation('linear'))
model.compile(loss='mse' , optimizer=opt , metrics=['mse'])


history = model.fit(train_X , train_y , epochs=500, steps_per_epoch=30 , verbose=1 ,validation_data=(test_X, test_y) ,shuffle=False,callbacks=callback)

Epoch 1/500
30/30 [==============================] - 10s 219ms/step - loss: 0.0838 - mse: 0.0838 - val_loss: 0.0765 - val_mse: 0.0765
Epoch 2/500
30/30 [==============================] - 6s 204ms/step - loss: 0.0742 - mse: 0.0742 - val_loss: 0.0654 - val_mse: 0.0654
Epoch 3/500
30/30 [==============================] - 6s 201ms/step - loss: 0.0599 - mse: 0.0599 - val_loss: 0.0462 - val_mse: 0.0462
Epoch 4/500
30/30 [==============================] - 6s 204ms/step - loss: 0.0403 - mse: 0.0403 - val_loss: 0.0305 - val_mse: 0.0305
Epoch 5/500
30/30 [==============================] - 6s 206ms/step - loss: 0.0281 - mse: 0.0281 - val_loss: 0.0216 - val_mse: 0.0216
Epoch 6/500
30/30 [==============================] - 6s 202ms/step - loss: 0.0206 - mse: 0.0206 - val_loss: 0.0162 - val_mse: 0.0162
Epoch 7/500
30/30 [==============================] - 6s 201ms/step - loss: 0.0163 - mse: 0.0163 - val_loss: 0.0135 - val_mse: 0.0135
Epoch 8/500
30/30 [==============================] - 6s 199ms/step -

**Loading Test Data...**

In [10]:
#TEST DATA

dataset_test_ok = pd.read_csv('/kaggle/input/dlproj/S_test.csv')
dataset_test_ok.head()

# Encoding Categorical Values to Numerical if any...
encoder1 = LabelEncoder()
dataset_test_ok.iloc[:,3] = encoder1.fit_transform(dataset_test_ok.iloc[:,3])

# Pre-processing
x_1 = dataset['weather'].values
x_2 = dataset['temperature'].values
x_3 = dataset['pressure'].values
x_4 = dataset['humidity'].values
x_5 = dataset['wind_speed'].values
x_6 = dataset['wind_direction'].values
x_7 = dataset['PM10_Concentration'].values
x_8 = dataset['NO2_Concentration'].values
x_9 = dataset['CO_Concentration'].values
x_10 = dataset['O3_Concentration'].values
x_11 = dataset['SO2_Concentration'].values
y = dataset['PM25_Concentration'].values

# Convert to [rows, columns] structure
x_1 = x_1.reshape((len(x_1), 1))
x_2 = x_2.reshape((len(x_2), 1))
x_3 = x_3.reshape((len(x_3), 1))
x_4 = x_4.reshape((len(x_4), 1))
x_5 = x_5.reshape((len(x_5), 1))
x_6 = x_6.reshape((len(x_6), 1))
x_7 = x_7.reshape((len(x_7), 1))
x_8 = x_8.reshape((len(x_8), 1))
x_9 = x_9.reshape((len(x_9), 1))
x_10 = x_10.reshape((len(x_10), 1))
x_11 = x_11.reshape((len(x_11), 1))
y = y.reshape((len(y), 1))
print ("x_1.shape" , x_1.shape) 
print ("x_2.shape" , x_2.shape) 
print ("y.shape" , y.shape)

#Normalization 
scaler = MinMaxScaler(feature_range=(0, 1))
x_1_scaled = scaler.fit_transform(x_1)
x_2_scaled = scaler.fit_transform(x_2)
x_3_scaled = scaler.fit_transform(x_3)
x_4_scaled = scaler.fit_transform(x_4)
x_5_scaled = scaler.fit_transform(x_5)
x_6_scaled = scaler.fit_transform(x_6)
x_7_scaled = scaler.fit_transform(x_7)
x_8_scaled = scaler.fit_transform(x_8)
x_9_scaled = scaler.fit_transform(x_9)
x_10_scaled = scaler.fit_transform(x_10)
x_11_scaled = scaler.fit_transform(x_11)
y_scaled = scaler.fit_transform(y)

# Horizontally stack columns
dataset_test_stacked = hstack((x_1_scaled, x_2_scaled,x_2_scaled, x_3_scaled, x_4_scaled, x_5_scaled,x_7_scaled,x_8_scaled, x_9_scaled,x_10_scaled,x_11_scaled, y_scaled))
print ("dataset_stacked.shape" , dataset_test_stacked.shape)

x_1.shape (3523, 1)
x_2.shape (3523, 1)
y.shape (3523, 1)
dataset_stacked.shape (3523, 12)


In [11]:
dataset_test_ok.head()

,Unnamed: 0,weather,temperature,pressure,humidity,wind_speed,wind_direction,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration,PM25_Concentration
0,2015-03-25 06:00:00,2.0,0.570588,155.0,0.76,0.013889,13.0,0.050505,0.029915,0.041591,0.226158,0.010020,24.0
1,2015-03-25 07:00:00,2.0,0.570588,159.0,0.75,0.013889,13.0,0.074495,0.064103,0.042851,0.182561,0.012024,30.0
2,2015-03-25 08:00:00,2.0,0.570588,159.0,0.75,0.013889,13.0,0.074495,0.064103,0.042851,0.182561,0.012024,30.0
3,2015-03-25 09:00:00,2.0,0.601961,172.0,0.71,0.034722,13.0,0.113636,0.175214,0.047262,0.119891,0.016032,37.0
4,2015-03-25 10:00:00,2.0,0.601961,174.0,0.71,0.037037,14.0,0.118687,0.226496,0.049783,0.098093,0.016032,39.0


# **Testing**

In [12]:
#Prediction

dataset_test_X = dataset_test_stacked
test_X_new, y = split_sequences(dataset_test_X, n_steps_in, n_steps_out)
print("dataset_test_X :",dataset_test_X.shape)
print("dataset_test_y :",y.shape)

y_pred = model.predict(test_X_new)

y_pred_inv = scaler.inverse_transform(y_pred)
print("y_pred :",y_pred.shape)
print("y_pred_inv :",y_pred_inv.shape)

from sklearn.metrics import mean_squared_error, mean_absolute_error

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y, y_pred)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)

dataset_test_X : (3523, 12)
dataset_test_y : (3367, 30)
106/106 [==============================] - 5s 41ms/step
y_pred : (3367, 30)
y_pred_inv : (3367, 30)
Mean Squared Error (MSE): 0.006804956263485068
Mean Absolute Error (MAE): 0.06208550218273405


**Save the Model...**

In [13]:
import pickle
pickle.dump(model,open("S_City.h5","wb"))